In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

In [2]:
import chess
from game_log_reader.playback import playback
from game_log_reader.uncertainty_playback_player import UncertaintyPlaybackPlayer
from game_log_reader.game_history_extended import GameHistoryExtended
import os
from tqdm import tqdm

In [7]:
path = 'game_logs/unranked_games/u4r1/Oracle'
save_path = 'game_logs/ranked_games_extended/batch_18 (0,125 - 100)'

if not os.path.exists(save_path):
    os.makedirs(save_path, mode=0o777)

print(len(os.listdir(path)), len(os.listdir(save_path)))

game_files = list(set(os.listdir(path)) - set(os.listdir(save_path)))
game_files.sort()

# game_files = ['571769.json']

49 294


In [8]:
bar = tqdm(game_files)

for game_file in bar:
    if not os.path.exists(os.path.join(path, game_file, f'game_{game_file}.log')):
        continue
    
    game_history = GameHistoryExtended.from_file(os.path.join(path, game_file, f'game_{game_file}.log'))

    if game_history.is_empty():
        continue
    for color in chess.COLORS:
        if game_history.get_player_name(color) in ['random', 'trout', 'attacker', 'Zubat']:
            continue
        bar.set_postfix({'game': game_file, 'player': game_history.get_player_name(color), 'opponent': game_history.get_player_name(not color)})
        player = UncertaintyPlaybackPlayer(game_id=game_file.split('.')[0], log_to_file=False, rc_disable_pbar=True)
        try:
            playback(game_history, player, color)
        except:
            pass
        game_history._pre_sense_uncertainty[color] = player.board_states_pre_sense
        game_history._post_sense_uncertainty[color] = player.board_states_post_sense
    os.makedirs(os.path.join(save_path, game_file), mode=0o777)
    game_history.save(os.path.join(save_path, game_file, f'game_{game_file}.log'))

100%|██████████| 49/49 [02:12<00:00,  2.70s/it, game=738959, player=Oracle, opponent=Zubat]


In [ ]:
game_files

In [ ]:
player.board_states_pre_sense

In [ ]:
player.board_states_post_sense